In [ ]:
import pandas as pd
import requests

In [ ]:
def request_melodi_filosifi(page:int=1):
    response = requests.get(
            "https://api.insee.fr/melodi/data/DS_FILOSOFI_CC" \
            f"?page={page}"
        )

    return response.json()

def get_melodi_filosifi_data() -> pd.DataFrame:
    data_dict = {}
    page = 1

    while True:
        json_response = request_melodi_filosifi(page)
        page += 1
        # loop will stop as soon as response does not include data (observations is empty)
        if observations:=json_response.get("observations"):
            for item in observations:
                geo, measure_id = item["dimensions"]["GEO"], item["dimensions"]["FILOSOFI_MEASURE"]
                # if value is defined, store it in data_dict
                try:
                    value = item["measures"]["OBS_VALUE_NIVEAU"]["value"]
                    # if geo is already a line, complete it
                    try:
                        data_dict[geo][measure_id] = value
                    # else create the line and complete it
                    except KeyError:
                        data_dict[geo] = {"GEO": geo}
                        data_dict[geo][measure_id] = value
                # if value is not defined, pass to next observation item
                except KeyError:
                    pass
        else:
            break
    
    # here we only want to keep values on communal scale
    df = pd.DataFrame([value for value in data_dict.values() if "2025-COM-" in value["GEO"]])
    return df

df = get_melodi_filosifi_data()